<a href="https://colab.research.google.com/github/soumya0422/FitPulse-Health-Anomaly-Detection-from-Fitness-Devices-/blob/main/Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install fastapi uvicorn streamlit pyngrok pandas requests prophet tsfresh scikit-learn matplotlib plotly --quiet

In [26]:
from pyngrok import ngrok
ngrok.set_auth_token("38DbZKBCEmeKeEQCvidwnGfJErz_UDgnfdabdR8xmC2R4Hh1")

In [27]:
%%writefile backend.py
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
import pandas as pd
import numpy as np
import traceback
from prophet import Prophet
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN

app = FastAPI(title="FitPulse – Health Anomaly Backend")

# ---------------- GLOBAL STATE ----------------
CLEAN_DF = None
FEATURE_DF = None
ALERTS_DF = None

# ---------------- ERROR HANDLER ----------------
@app.exception_handler(Exception)
async def global_exception_handler(request, exc):
    trace = traceback.format_exc()
    print(trace)
    return JSONResponse(
        status_code=500,
        content={"error": "Backend crashed", "trace": trace}
    )

# ---------------- MODULE 1 – PREPROCESSING ----------------
@app.post("/preprocess")
def preprocess(file: UploadFile = File(...)):
    global CLEAN_DF
    df = pd.read_csv(file.file)
    REQUIRED = ["Id", "date", "avg_heart_rate", "daily_steps", "hours_sleep"]
    missing = [c for c in REQUIRED if c not in df.columns]
    if missing:
        return JSONResponse(status_code=400, content={"error": f"Missing {missing}"})
    df = df.rename(columns={
        "Id": "user_id",
        "avg_heart_rate": "heart_rate",
        "daily_steps": "steps",
        "hours_sleep": "sleep"
    })
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df = df.dropna(subset=["date"])
    for col in ["heart_rate", "steps", "sleep"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")
    df["heart_rate"].fillna(df["heart_rate"].median(), inplace=True)
    df["steps"].fillna(0, inplace=True)
    df["sleep"].fillna(df["sleep"].median(), inplace=True)
    df = (
        df.set_index("date")
          .groupby("user_id")[["heart_rate", "steps", "sleep"]]
          .resample("D")
          .mean()
          .reset_index()
    )
    CLEAN_DF = df
    df.to_csv("clean_data.csv", index=False)
    return {"status": "success", "rows": len(df)}

# ---------------- MODULE 2 – FEATURE EXTRACTION + MODELING ----------------
@app.post("/module2")
def module2():
    global CLEAN_DF, FEATURE_DF
    if CLEAN_DF is None:
        return JSONResponse(status_code=400, content={"error": "Run Module 1 first"})
    df = CLEAN_DF.sort_values(["user_id", "date"])
    df["hr_7d_mean"] = df.groupby("user_id")["heart_rate"].transform(lambda x: x.rolling(7, min_periods=1).mean())
    df["hr_7d_std"] = df.groupby("user_id")["heart_rate"].transform(lambda x: x.rolling(7, min_periods=1).std().fillna(0))
    df["steps_7d_mean"] = df.groupby("user_id")["steps"].transform(lambda x: x.rolling(7, min_periods=1).mean())
    FEATURE_DF = df.copy()
    # Prophet forecasting
    plot = []
    try:
        p_df = df.groupby("date")["heart_rate"].mean().reset_index().rename(columns={"date":"ds","heart_rate":"y"})
        if len(p_df) > 10:
            m = Prophet()
            m.fit(p_df)
            f = m.predict(p_df)
            p_df["yhat"] = f["yhat"]
            plot = p_df.tail(30).to_dict("records")
    except:
        pass
    X = FEATURE_DF[["heart_rate","steps","sleep","hr_7d_mean"]].fillna(0)
    X = StandardScaler().fit_transform(X)
    db = DBSCAN(eps=1.2, min_samples=5)
    FEATURE_DF["cluster"] = db.fit_predict(X)
    FEATURE_DF["cluster_anomaly"] = FEATURE_DF["cluster"] == -1
    FEATURE_DF.to_csv("feature_data.csv", index=False)
    return {"status": "success", "sample_plot": plot}

# ---------------- MODULE 3 – ANOMALY DETECTION ----------------
@app.post("/module3")
def module3():
    global FEATURE_DF, ALERTS_DF
    if FEATURE_DF is None:
        return JSONResponse(status_code=400, content={"error": "Run Module 2 first"})
    rows = []
    for _, r in FEATURE_DF.iterrows():
        if r["heart_rate"] > 120:
            rows.append((r["user_id"], r["date"], "heart_rate_high"))
        if r["heart_rate"] < 40:
            rows.append((r["user_id"], r["date"], "heart_rate_low"))
        if r["sleep"] < 4 or r["sleep"] > 12:
            rows.append((r["user_id"], r["date"], "sleep_abnormal"))
        if r["cluster_anomaly"]:
            rows.append((r["user_id"], r["date"], "cluster_outlier"))
    df = pd.DataFrame(rows, columns=["user_id","date","metric"])
    alerts = df.groupby(["user_id","metric"]).agg(count=("date","count")).reset_index()
    alerts["severity"] = alerts["count"].apply(lambda x: "High" if x>=5 else "Medium" if x>=3 else "Low")
    ALERTS_DF = alerts
    alerts.to_csv("module3_alerts.csv", index=False)
    return {"status":"success","alerts":alerts.to_dict("records")}

# ---------------- DASHBOARD DATA ----------------
@app.get("/dashboard")
def dashboard():
    global CLEAN_DF
    if CLEAN_DF is None:
        return JSONResponse(status_code=400, content={"error":"Run /preprocess first"})
    heart_rate_df = CLEAN_DF.groupby("date")["heart_rate"].mean().reset_index().rename(columns={"date":"timestamp"})
    steps_df = CLEAN_DF.groupby("date")["steps"].mean().reset_index().rename(columns={"date":"timestamp","steps":"step_count"})
    summary = {
        "avg_heart_rate": round(CLEAN_DF["heart_rate"].mean(),2),
        "avg_steps": int(CLEAN_DF["steps"].mean()),
        "total_records": len(CLEAN_DF)
    }
    return {
        "status":"success",
        "heart_rate_series": heart_rate_df.to_dict("records"),
        "steps_series": steps_df.to_dict("records"),
        "summary":summary
    }

# ---------------- MODULE 4 – INSIGHTS ----------------
@app.post("/module4")
def module4():
    if ALERTS_DF is None or ALERTS_DF.empty:
        return JSONResponse(status_code=400, content={"error":"Run Module 3 first"})
    insights = [
        {"user_id":r["user_id"],"severity":r["severity"],
         "insight":f"User {r['user_id']} has {r['severity']} risk due to {r['metric']}"}
        for _, r in ALERTS_DF.iterrows()
    ]
    return {"status":"success","insights":insights}

# ---------------- RUN Uvicorn + Ngrok ----------------
import uvicorn, threading
public_url = ngrok.connect(8000)
print("FastAPI public URL:", public_url)
threading.Thread(target=uvicorn.run, kwargs={"app":"backend:app","host":"0.0.0.0","port":8000}).start()


Overwriting backend.py


In [30]:
%%writefile app.py
import streamlit as st
import requests
import pandas as pd
import plotly.express as px
from datetime import datetime

BACKEND_URL = "http://localhost:8000"  # Will work with ngrok URL dynamically if needed

st.set_page_config(page_title="FitPulse – Health Anomaly Dashboard", layout="wide")

# ---------------- Helper ----------------
def handle_response(res, label=""):
    st.write(f"🔍 {label} status:", res.status_code)
    content_type = res.headers.get("content-type","")
    if content_type.startswith("application/json"):
        return res.json()
    st.error("❌ Backend returned NON-JSON response")
    st.code(res.text)
    return None

# ---------------- Sidebar ----------------
st.sidebar.header("📂 Upload Fitness Data")
uploaded_file = st.sidebar.file_uploader("Upload CSV/JSON", type=["csv","json"])

if uploaded_file:
    with st.spinner("Preprocessing data..."):
        try:
            res = requests.post(f"{BACKEND_URL}/preprocess", files={"file":uploaded_file})
        except requests.exceptions.RequestException as e:
            st.sidebar.error(f"Connection failed: {e}")
            st.stop()
        data = handle_response(res,"Preprocess")
        if not data: st.stop()
        if res.status_code == 200:
            st.sidebar.success("✅ Data processed")
            st.sidebar.metric("Rows Loaded", data.get("rows"))
        else:
            st.sidebar.error(data.get("error"))
            st.stop()

# ---------------- Tabs ----------------
tab1, tab2, tab3, tab4 = st.tabs(["📊 Overview","📈 Health Dashboard","🚨 Anomaly Detection","🧠 Insights & Reports"])

# ===== TAB1: Overview =====
with tab1:
    st.subheader("Dataset Overview")
    try:
        df = pd.read_csv("clean_data.csv")
        st.dataframe(df.head(50), use_container_width=True)
        c1,c2,c3 = st.columns(3)
        c1.metric("Users", df["user_id"].nunique())
        c2.metric("Days", df["date"].nunique())
        c3.metric("Avg HR", round(df["heart_rate"].mean(),1))
    except FileNotFoundError:
        st.info("⬅️ Upload data to see overview")
        st.stop()

# ===== TAB2: Health Dashboard =====
with tab2:
    st.subheader("📊 Health Dashboard")
    if st.button("Load Dashboard"):
        with st.spinner("Fetching dashboard data..."):
            try:
                res = requests.get(f"{BACKEND_URL}/dashboard")
            except requests.exceptions.RequestException as e:
                st.error(f"Connection error: {e}")
                st.stop()
            data = handle_response(res,"Dashboard")
            if not data: st.stop()
            hr_df = pd.DataFrame(data["heart_rate_series"])
            steps_df = pd.DataFrame(data["steps_series"])
            if not hr_df.empty:
                fig = px.line(hr_df, x="timestamp", y="heart_rate", title="❤️ Heart Rate Over Time")
                st.plotly_chart(fig, use_container_width=True)
            if not steps_df.empty:
                fig = px.bar(steps_df, x="timestamp", y="step_count", title="🚶 Steps Over Time")
                st.plotly_chart(fig, use_container_width=True)

# ===== TAB3: Anomaly Detection =====
with tab3:
    st.subheader("🚨 Anomaly Detection")
    if st.button("Detect Anomalies"):
        with st.spinner("Scanning health metrics..."):

            # 1️⃣ Run Module 2 first (feature extraction + clustering)
            try:
                res2 = requests.post(f"{BACKEND_URL}/module2")
            except requests.exceptions.RequestException as e:
                st.error(f"Connection failed (Module 2): {e}")
                st.stop()

            data2 = handle_response(res2, "Module 2")
            if not data2: st.stop()
            if res2.status_code != 200:
                st.error("Module 2 failed: " + data2.get("error", "Unknown"))
                st.stop()

            # 2️⃣ Now run Module 3 (anomaly detection)
            try:
                res3 = requests.post(f"{BACKEND_URL}/module3")
            except requests.exceptions.RequestException as e:
                st.error(f"Connection failed (Module 3): {e}")
                st.stop()

            data3 = handle_response(res3, "Module 3")
            if not data3: st.stop()

            if res3.status_code == 200:
                alerts = pd.DataFrame(data3["alerts"])
                if alerts.empty:
                    st.success("No anomalies detected 🎉")
                else:
                    st.dataframe(alerts, use_container_width=True)
                    fig = px.bar(
                        alerts,
                        x="metric",
                        y="count",
                        color="severity",
                        title="Detected Health Alerts"
                    )
                    st.plotly_chart(fig, use_container_width=True)
            else:
                st.error(data3.get("error"))


# ===== TAB4: Insights & Reports =====
with tab4:
    st.subheader("🧠 Health Risk Insights")
    if st.button("Generate Insights"):
        with st.spinner("Generating insights..."):
            try:
                res = requests.post(f"{BACKEND_URL}/module4")
            except requests.exceptions.RequestException as e:
                st.error(f"Connection failed: {e}")
                st.stop()
            data = handle_response(res,"Module 4")
            if not data: st.stop()
            if res.status_code == 200:
                for i in data["insights"]:
                    st.warning(f"User {i['user_id']} — {i['severity']} Risk\n{i['insight']}")
            else:
                st.error(data.get("error"))
    st.divider()
    st.subheader("⬇️ Download Reports")
    try:
        st.download_button("Download Clean Data", open("clean_data.csv","rb"), file_name="clean_data.csv")
        st.download_button("Download Anomaly Report", open("module3_alerts.csv","rb"), file_name="health_alerts.csv")
    except FileNotFoundError:
        st.info("Run analysis to generate reports")

Overwriting app.py


In [29]:
# ===============================
# Launch Streamlit via Ngrok
# ===============================
import threading, time
def run_streamlit():
    !streamlit run app.py --server.port 8501 --server.address 0.0.0.0
threading.Thread(target=run_streamlit).start()
time.sleep(5)
streamlit_url = ngrok.connect(8501)
print("Streamlit public URL:", streamlit_url)



2026-01-13 20:49:44.614 Port 8501 is already in use
Streamlit public URL: NgrokTunnel: "https://renowned-zuri-coleopterous.ngrok-free.dev" -> "http://localhost:8501"
